<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="0_Dataoving3.ipynb">Dataøving 3</a>
    </div>
    <ul class="nav navbar-nav">
        <li class="active"><a href="Oppgave1.ipynb">Oppgave 1 - Introduksjon til spektral lekkasje</a></li>
        <li><a href="Oppgave2.ipynb">Oppgave 2 - Zero Padding og DTFT</a></li>
        <li><a href="Oppgave3.ipynb">Oppgave 3 - Vindusfunksjoner</a></li>
        <li><a href="Oppgave4.ipynb">Oppgave 4 - Spektrogram: <i>Tilstandsovervåking av maskineri del 2</i></a></li>
    </ul>
  </div>
</nav>

# Hva er spektral lekkasje?

**Tema:**
- Vinduslengde og signalperioder
- Spektral lekkasje

**Læringsmål:**
- Forstå årsaken til spektral lekkasje
- Forsåt betydningen av vinduslengde for utregnet Diskrét Fouriertransformasjon

**Bibliotek og notebook-konfigurasjon:**

In [1]:
from numpy import sin, cos, pi, exp, empty, mean, absolute, angle # Sentrale matematiske funksjoner
from numpy.fft import fft, ifft             # DFT og IDFT
import numpy as np                          # Importer funksjonalitet fra numpy biblioteket med prefiks "np"
import scipy.signal as sig                  # Importerer signalbehandlingsmodulen til scipy
import matplotlib.pyplot as plt             # Importer pyplot modulen i matplotlib med prefiks "plt"
import csv                                  # Håndtering av .csv fil

%matplotlib ipympl

## Introduksjon

I dataøving 3 arbeidet vi med Diskrét Fouriertransformasjon fra et teoretisk ståsted, og utforsket hvordan hvert element i en DFT (sammen med en komplekskonjugert "tvilling") representerer nøyaktig én sinussekvens i tidsdomenet. Dessverre gjelder dette *kun* for sinussekvenser med den egenskapen at de er periodisk over samme antall sampler $N$ som går inn i fouriertransformasjonen, noe vi kaller **vinduslengden**. 

I praktiske sammenhenger der frekvensanalyse av et ukjent signal skal utføres kan man nesten garantere at signalet *ikke* er periodisk over vinduslengden, og det oppstår noe som kalles **spektral lekkasje**. I dette tilfellet vil fouriertransformasjon gi sinusbølge-komponentene til et signal som *er* periodisk med vinduslengden. Effekten av dette gjør seg selv svært synlig dersom man prøver å rekonstruere et tidskontinuerlig signal fra en DFT.

### Eksempel: rekonstruert sinusbølge.

Signalet $x(t) = \cos \left( 2\pi \cdot 1.6 \cdot t \right)$ samples med en samplingsfrekvens $f_s = 16$, noe som gir signalet $x[n] = \cos \left( \frac{\pi}{5} \cdot n \right)$. Et utsnitt på $N=16$ sampler, tilsvarende $\frac{N}{f_s} = \frac{16}{16} = 1$ sekund, benyttes til å fouriertransformere signalet. Det man ender opp med er komposisjonen til et nytt signal $y(t)$ som **er** periodisk over vinduslengden (altså ett sekund), og kun har riktig verdi akkurat i de 16 samplingstidspunktene som er brukt.

![](Figurer/SpektralLekkasje.png)

Å beregne en 16-punkts DFT kan med andre ord betraktes som å "finne en måte å gjengi akkurat disse 16 utvalgte samplene, kun med bruk av sinussekvenser som er periodiske over 16 sampler". Det følger dermed at det resulterende sammensatte signalet også vil være periodisk over 16 sampler. Mer generelt betyr det at en $N$-punkts DFT dekomponerer et signalutklipp i en rekke sinussekvenser med normalisert vinkelfrekvens $\hat{\omega}_k = 2\pi \frac{k}{N}, \ \ \ k \in \{0, 1, 2, \ldots N/2\}$.  

Resultatet av dette er en representasjon av sinussekvensen $x[n]$ i frekvensplanet som er fordelt utover et større antall frekvenskomponenter, da ingen av komponentene man har "lov" til å bruke har samme frekvens som sinussekvensen $x[n]$. Figuren nedenfor viser frekvensinnholdet slik det framstår ut ifra $|X[k]|$ utregnet med 16-punkts DFT, representert sammen med det reelle frekvensinnholdet til $x(t)$. 

![](Figurer/SpektralLekkasje2.png)

Det figuren illustrerer er at frekvensinnholdet til sinusbølgen ikke kan representeres med noen av de spesifikke frekvensene DFT-sekvensen er sammensatt av. Dermed oppstår en fordeling av signalet utover de 16 tilgjengelige frekvenskomponentene, og det er dette som kalles spektral lekkasje.

Kodecellen nedenfor viser et eksempel på programkode som genererer $N=16$ sampler av signalet $x[n]$, regner ut DFT-sekvensen $X[k]$ basert på de 16 samplene, og viser resultatet i et stolpediagram.

In [2]:
# Lukk evt. eksisterende figur 1 og lag en ny figur 1.
# Dette for å unngå akkumulering av figurer i bakgrunnen som tar opp minne.
plt.close(1) 
plt.figure(1)

# Parametre
N = 16      # Antall sampler av sinusbølge i DFT
f_0 = 1.6   # Frekvens sinusbølge i Hz
f_s = 16    # Samplingsfrekvens

# Utregning av sinussekvens og kalkulasjon av DFT
n = np.arange(N)
x_n = cos(2*pi*f_0/f_s*n)
X_k = fft(x_n)

# Funksjonen "stem()" brukes til å lage et stolpediagram av DFT-sekvensens absoluttverdi |X[k]|
# Påfølgende linjer utfører tilpasninger av figuren
plt.stem(n, # Samplenummer (heltall fom. 0 tom. 15)
         np.absolute(X_k),   # DFT-sekvensens absoluttverdi |X[k]|
         linefmt='-',        # Linjestil stolper
         markerfmt='o',      # Punktstil for stem-markere. Default er 'o' (stor prikk)
         basefmt='grey',     # Farge på y=0 aksen
         use_line_collection=True # Hvordan "stem()" skal håndtere dataene. "True" er sterkt anbefalt.
         )
plt.xlabel("DFT-indeks 'k'") # Merknad på x-aksen
plt.ylabel(r"$|X[k]|$")      # Merknad på y-aksen
plt.xlim([0, N])             # Grenseverdier i x-retning
plt.ylim(ymin=0)             # y-aksen starter i y=0
plt.grid(True)               # Aktiver rutenett 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Oppgave:

Denne oppgaven tar utgangspunkt i et signal $x(t)$ bestående av to sinuskomponenter:

$$x(t) = 2\cos\left(19\pi\cdot t + \frac{\pi}{3}\right) + \cos\left(23\pi \cdot t \right)$$

Signalet samples med samplingsfrekvens $f_s = 80 Hz$ for å prodsere signalet $x[n]$. 

## a)
Skriv kode som genererer $N=32$ sampler av signalet $x[n]$. 

In [3]:
### BEGIN SOLUTION
fs = 80
N = 160
n = np.arange(N)
xn = 2*cos(19*pi*n/fs + pi/3) + cos(23*pi*n/fs)
f = fs/N*n

Xk = fft(xn)
plt.close(2); plt.figure(2)
plt.stem(f, np.abs(Xk), markerfmt='.', basefmt='grey', use_line_collection=True)
plt.xlabel('Frekvens (Hz)')
### END SOLUTION

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Frekvens (Hz)')

## b)
Utvid koden i cellen over til å regne ut DFT-sekvensen $X[k]$, og generer et stolpediagram som viser amplitudespekteret $|X[k]|$ til signalet $x[n]$. 

## c)

Gjør ytterligere endringer på koden slik at figuren viser tosidig frekvensspekter, og x-aksen nå viser frekvens (i Hz), og ikke indeksverdien 'k'.

## d) 
Eksperimentér videre med vinduslengder $N$. Finn en vinduslengde vinduslengde $N$ der de to sinuskomponentene vises som to adskilte topper i amplitudespekteret.

Ved å prøve seg frem med ulike verdier av $N$ ser det ut til at vinduslengde $N=54$ er blant de første tilfellene der vi tydelig ser to ulike sinuskomponenter.

### e)

Klarer du å finne en vinduslengde $N$ slik at spektral lekkasje ikke forekommer? I såfall, hva er den?

De to sinuskomponentene har følgende digitale frekvenser:
\begin{align}
\hat{\omega}_1 &= \frac{19\pi}{f_s} = 2\pi \cdot \frac{9.5}{80} = 2\pi \cdot \frac{19}{160}\\
\hat{\omega}_2 &= \frac{23\pi}{f_s} = 2\pi \cdot \frac{11.5}{80}= 2\pi \cdot \frac{23}{160}
\end{align}
Dette betyr at dersom vi observerer signalet over $t=2s$ eller $N=2\cdot f_s = 160$ sampler, vil begge sinuskomponentene fullføre et heltall antall perioder i løpet av vinduslengden $N=160$.